In [103]:
import random
import numpy as np
from  itertools import combinations
from scipy import stats
import statsmodels
import statsmodels.stats


In [146]:
import statsmodels.stats.multitest as sm
sm.multipletests([.05, .05], method = 'fdr_bh')

(array([ True,  True]), array([0.05, 0.05]), 0.025320565519103666, 0.025)

In [2]:
import pandas as pd

In [3]:
def resample(l):
    final = []
    for i in range(len(l)):
        final.append(random.choice(l))
    return final

def repeat_resample(sample_a, sample_b, num_iter = 1000):
    difference_in_means = []#keep track of the difference in heights for each experiment
    for i in range(num_iter):
        resample_a = resample(sample_a)
        resample_b = resample(sample_b)
        difference = np.mean(resample_a) - np.mean(resample_b)
        difference_in_means.append(difference)
    return difference_in_means

def comb_samples_to_pop(*args):
    all_ = []
    for i in args:
        all_.extend(i)
    return all_


In [113]:
def anova_omnibus( *args, num_iter = 1000, verbose = False):
    pop = comb_samples_to_pop(*args)
    all_means = []
    for  i in args:
        all_means.append(np.mean(i))
    var_between_groups = np.var(all_means)
    if verbose:
        print(all_means)
        print(var_between_groups)
    resampled_vars = []
    for i in range(num_iter):
        random.shuffle(pop)
        init = 0
        means_of_groups = []
        for i in args:
            s = pop[init:len(i) + init]
            means_of_groups.append(np.mean(s))
            init = len(i)
        resampled_vars.append(np.var(means_of_groups))
    diff = [var_between_groups - x for x in resampled_vars]
    p_value =  1 - len([x for x in diff if x > 0])/len(diff)
    return p_value
    

    
s1 = [random.gauss(0,1) for x in range(100)]
s2 = [random.gauss(0,1) for x in range(100)]
s3 = [random.gauss(.3,1) for x in range(100)]
p_mine = anova_omnibus(s1, s2, s3, verbose = True, num_iter = 100) 
F, p = stats.f_oneway(s1, s2, s3 )
print(round(p_mine,2), round(p, 2) )
resamps = repeat_resample(s1, s2)
p_value1 = len([x for x in resamps if x < 0])/len(resamps)
p_value2 = len([x for x in resamps if x > 0])/len(resamps)
resamps = repeat_resample(s2, s3)
p_value3 = len([x for x in resamps if x > 0])/len(resamps)

print(p_value, p_value1, p_value2,
      p_value3)
method = 'bonferroni'
#method = 'fdr_by'
p_adjusted = multipletests([p_value1, p_value2, p_value3], method=method)
p_adjusted

[0.3196406600709537, 0.003332635805666363, 0.28101497095264366]
0.01985002103635024
0.01 0.05
0.0 0.014 0.986 0.023


(array([ True, False, False]),
 array([0.042, 1.   , 0.069]),
 0.016952427508441503,
 0.016666666666666666)

In [107]:
def get_data():
    perm_mold = [45.5, 45.3, 45.4, 44.4, 44.6, 43.9, 44.6, 44.0]
    die_casting = [44.2, 43.9, 44.7, 44.2, 44, 43.8, 44.6, 43.1]
    plast_mold = [46, 45.9, 44.8, 46.2, 45.1, 45.5]
    df = pd.DataFrame([perm_mold, die_casting, plast_mold ],  index = ['perm_mold', 'die_casting', 'plast_mold'])
    return df
data = get_data()
s1 = data.loc['perm_mold']
s2 = data.loc['die_casting']
s3 = data.loc['plast_mold'].dropna()
p_value = anova_omnibus(s1, s2, s3)
p_value
#let compare
resamps = repeat_resample(s1, s2)
p_value1 = len([x for x in resamps if x < 0])/len(resamps)
resamps = repeat_resample(s1, s3)
p_value2 = len([x for x in resamps if x > 0])/len(resamps)
resamps = repeat_resample(s2, s3)
p_value3 = len([x for x in resamps if x > 0])/len(resamps)

print(p_value, p_value1,stats.ttest_ind(s1, s2)[1], p_value2, p_value1,stats.ttest_ind(s1, s3)[1],
      p_value3)
method = 'bonferroni'
#method = 'fdr_by'
p_adjusted = multipletests([p_value1, p_value, p_value3], method=method)
p_adjusted


0.0 0.002 0.03762203344387319 0.002 0.002 0.01882889108209598 0.0


(array([ True,  True,  True]),
 array([0.006, 0.   , 0.   ]),
 0.016952427508441503,
 0.016666666666666666)

In [76]:
.05 * 3

0.15000000000000002

In [7]:
F, p = stats.f_oneway(data.loc['perm_mold'], data.loc['die_casting'], 
                      data.loc['plast_mold'].dropna() )
p

0.0003336160734123371

In [80]:
s1 = [random.gauss(0,1) for x in range(100)]
s2 = [random.gauss(0,1) for x in range(100)]
s3 = [random.gauss(.3,1) for x in range(100)]
pm = anova_omnibus(s1, s2, s3, verbose = False, num_iter = 100) 



Ttest_indResult(statistic=-1.3096970309327995, pvalue=0.19181594508231287)

In [85]:
print(stats.ttest_ind(s1, s2))
resamps =  repeat_resample(s1, s2, num_iter = 1000)
1 - len([x for x in resamps if x < 0])/len(resamps)



Ttest_indResult(statistic=-1.3096970309327995, pvalue=0.19181594508231287)


0.11099999999999999

In [102]:
from statsmodels.sandbox.stats.multicomp import multipletests
p_adjusted = multipletests([.05, .05, .05], method='fdr_by')
print(p_adjusted)


(array([False, False, False]), array([0.09166667, 0.09166667, 0.09166667]), 0.016952427508441503, 0.016666666666666666)


In [183]:
def fdr(p_values, alpha = .05):
    final = []
    s = sorted(p_values)
    for counter, i in enumerate(s):
        a_p = round(alpha * (counter + 1)/len(p_values),2)
        tp = round(i * len(p_values)/(counter + 1),4)
        if i < a_p:
            final.append(True)
        else:
            final.append(False)
        print(i, a_p, tp)
    return final
method = 'bonferroni'
method = 'fdr_bh'
p_values = [.08, .001, .003, .05, .06,  .088]
p_adj = sm.multipletests(p_values, method = method)
print(p_adj)
print(fdr(p_values))

(array([False,  True,  True, False, False, False]), array([0.088, 0.006, 0.009, 0.088, 0.088, 0.088]), 0.008512444610847103, 0.008333333333333333)
0.001 0.01 0.006
0.003 0.02 0.009
0.05 0.03 0.1
0.06 0.03 0.09
0.08 0.04 0.096
0.088 0.05 0.088
[True, True, False, False, False, False]


In [115]:
method = 'bonferroni'
method = 'fdr_by'
p_adjusted = multipletests([.05, .05, .05], method=method)
p_adjusted

(array([False, False, False]),
 array([0.09166667, 0.09166667, 0.09166667]),
 0.016952427508441503,
 0.016666666666666666)